# Iron Supported, Nitrogen-doped, Graphene (And Related Systems) <a name="head"></a>
# Descriptor Scaling
***

## Import Modules <a name="import-modules"></a>

In [23]:
import cufflinks as cf

In [24]:
# iPython Settings
# %load_ext autoreload
# %autoreload 2

# sys.path.insert(0, "/home/raul_surface/nglview_temp/nglview")

# Python Modules

import sys
sys.path.insert(0, "..")

import numpy as np
import pandas as pd
pd.options.mode.chained_assignment = None
from ase.visualize import view
import plotly.graph_objs as go
import plotly.plotly as py
import cufflinks as cf

from sklearn.linear_model import LinearRegression

# My Modules
from colors.colors import color_list

# Local Imports
from an_data_processing import load_df

## Script Inputs

In [25]:
marker_size = 12

line_x_array = [-0.5, 3.5]
# color_list = ["red", "blue"]

border_color_list = [
    "red",
    "blue",
    "green",
    "magenta",
    ]

## Construct Dataframe

In [26]:
df = load_df(from_file=True, root_dir="..")

df = df[df["adsorbate"] != "h2o"]
# df = df[df["adsorbate"] != "oh"]
# df = df[df["system"] != "Fe_slab"]
# df = df[df["system"] != "graphene"]

df = df[[
    "ads_e",
    "adsorbate",
    "site",
    "spinpol",
    "system",
    "notes"
    ]]

df.set_index(["site", "system", "spinpol"], inplace=True)

df = df.pivot(index=None, columns="adsorbate")

df_1 = df.xs("ads_e", axis=1, drop_level=True)
df_2 = df.xs("notes", axis=1, drop_level=True)

# df_2 = df_2.rename(index=str, columns={
df_2 = df_2.rename(columns={
    "o": "o-notes",
    "oh": "oh-notes",
    "ooh": "ooh-notes",
    })

df["index"] = df.index.values
df["index"] = df["index"].astype(str)

df = pd.concat([df_1, df_2], axis=1, join_axes=[df_1.index])

df_master = df.copy()
# text_i = df["oh-notes"].tolist()

df_master["notes"] = df_master["o-notes"] + " | " + df_master["oh-notes"] + " | " + df_master["ooh-notes"]

Attempting to load df from pickle


In [27]:
df

adsorbate                                  o        oh       ooh  \
site            system     spinpol                                 
C-C-bridged     N_graph_Fe NaN      2.559569  0.926760  4.643908   
C-N-bridged     N_graph_Fe NaN      3.073313  2.320686  3.587794   
C-ontop-ontop   N_graph_Fe NaN      1.318423  0.598220  3.250884   
C-ontop-trifold N_graph_Fe NaN      2.318185  0.924204  4.005248   
N-ontop         N_graph_Fe NaN      3.073390  2.324954  4.564022   
bridge          Fe_slab    False   -0.547471  0.053767  0.121138   
                           True    -0.585168 -0.301352 -0.085686   
                graph_Fe   False    2.659350  0.937604  4.644292   
                           True     2.550360  0.809974  4.536813   
bridged         graphene   False    3.505933  3.191883  5.314396   
                           True     3.506062  3.192040  5.250435   
ontop           Fe_slab    False    0.995073  0.053646  0.747704   
                           True     0.598335  0.129865  3.247015   
                graphene   False    4.163351  3.108575  5.305085   
                           True     4.163494  3.108821  5.305304   
ontop-ontop     graph_Fe   False    3.091699  1.605304  5.190799   
                           True     2.829212  1.364498  4.584199   
ontop-trifold   graph_Fe   False    2.822813  0.939768  4.425842   
                           True     2.628735  0.958965  3.893719   
ring-center     N_graph_Fe NaN      1.318204  0.607377  4.002371   
                graph_Fe   False    2.823435       NaN  4.690441   
                           True     4.857884  2.345813  4.418953   
                graphene   False    3.505860  3.134048  5.237423   
                           True     5.149554  3.134219  5.238015   
trifold         Fe_slab    False   -0.547445  0.053671  0.747959   
                           True    -0.017167 -0.297904       NaN   

adsorbate                                            o-notes  \
site            system     spinpol                             
C-C-bridged     N_graph_Fe NaN                                 
C-N-bridged     N_graph_Fe NaN                                 
C-ontop-ontop   N_graph_Fe NaN                                 
C-ontop-trifold N_graph_Fe NaN                                 
N-ontop         N_graph_Fe NaN                                 
bridge          Fe_slab    False                               
                           True                                
                graph_Fe   False                     Gr-slip   
                           True                      Gr-slip   
bridged         graphene   False                               
                           True                                
ontop           Fe_slab    False                               
                           True                                
                graphene   False                               
                           True                                
ontop-ontop     graph_Fe   False                         NaN   
                           True                          NaN   
ontop-trifold   graph_Fe   False                         NaN   
                           True                          NaN   
ring-center     N_graph_Fe NaN                                 
                graph_Fe   False    Reconfig (ontop-trifold)   
                           True                     Desorbed   
                graphene   False                               
                           True                     Desorbed   
trifold         Fe_slab    False                               
                           True                                

adsorbate                                           oh-notes    ooh-notes  
site            system     spinpol                                         
C-C-bridged     N_graph_Fe NaN                                   Desorbed  
C-N-bridged     N_graph_Fe NaN                      Desorbed              

In [28]:
def point_plot_settings(i_ind, index, row, hover_text_row="notes"):
    """
    """
    #| - point_plot_settings
#     print(row)
    
    system = row.name[1]

    if system == "N_graph_Fe":
        marker_line = border_color_list[0]
    if system == "graph_Fe":
        marker_line = border_color_list[1]
    if system == "graphene":
        marker_line = border_color_list[2]
    if system == "Fe_slab":
        marker_line = border_color_list[3]

    hover_text = row[hover_text_row]
    
    if i_ind == 0:
        show_leg = True
    else:
        show_leg = False

    if row["ooh-notes"] == "Desorbed":
        marker_sym = "x"
    elif row["oh-notes"] == "Desorbed":
        marker_sym = "x"
    elif row["o-notes"] == "Desorbed":
        marker_sym = "x"

    elif row["ooh-notes"] == "Dissociated":
        marker_sym = "diamond"
    elif row["oh-notes"] == "Dissociated":
        marker_sym = "diamond"
    elif row["o-notes"] == "Dissociated":
        marker_sym = "diamond"
       
    else:
        marker_sym = "circle"    
    

    out_dict = {
        "marker_symbol": marker_sym,
        "show_legend": show_leg,
        "hover_text": hover_text,
        "marker_line": marker_line,
#         "marker_sym":,
    }
    
    return(out_dict)
    #__|

# Linear Regression to Adsorbed Species

# Plotting Scaling Data Series

In [29]:
trace_list = []

## OOH vs OH

In [30]:
tmp_bool = False
for i_ind, (index, row) in enumerate(df_master.iterrows()):
    plot_settings_dict = point_plot_settings(i_ind, index, row)
    
    show_leg = False
    if tmp_bool is False:
        if plot_settings_dict["marker_symbol"] == "circle":
            tmp_bool = True
            show_leg = True

            trace_i = go.Scatter(
                y = [row["ooh"]],
                x = [row["oh"]],
                hoverinfo="text",
                text=plot_settings_dict["hover_text"],
                mode='markers',
                name = "*OOH scaling",
                legendgroup="*OOH scaling",
                showlegend=show_leg,
                marker=dict(
                    size=marker_size,
                    symbol=plot_settings_dict["marker_symbol"],
                    color = color_list[0],
                )
            )
            trace_list.append(trace_i)

    trace_i = go.Scatter(
        y = [row["ooh"]],
        x = [row["oh"]], 
        hoverinfo="text",
        text=plot_settings_dict["hover_text"],
        mode='markers',
        name = "*OOH scaling",
        legendgroup="*OOH scaling",
#         showlegend=show_leg,
        showlegend=False,
        marker=dict(
            size=marker_size,
            symbol=plot_settings_dict["marker_symbol"],
            color = color_list[0],

            line = dict(
                color = plot_settings_dict["marker_line"],
                width = 2,
            )
        )
    )
    trace_list.append(trace_i)

In [31]:
row["oh"]

-0.2979038869470245

## O vs OH

In [32]:
tmp_bool = False
for i_ind, (index, row) in enumerate(df_master.iterrows()):
    plot_settings_dict = point_plot_settings(i_ind, index, row)
 
    show_leg = False
    if tmp_bool is False:
        if plot_settings_dict["marker_symbol"] == "circle":
            tmp_bool = True
            show_leg = True

            trace_i = go.Scatter(
                y = [row["o"]],
                x = [row["oh"]],
                hoverinfo="text",
                text=plot_settings_dict["hover_text"],
                mode='markers',
                name = "*O scaling",
                legendgroup="*O scaling",
                showlegend=True,
                marker=dict(
                    size=marker_size,
                    symbol=plot_settings_dict["marker_symbol"],
                    color = color_list[1],
                )
            )            
            trace_list.append(trace_i)

    trace_i = go.Scatter(
        y = row[["o"]],
        x = row[["oh"]],
        hoverinfo="text",
        text=plot_settings_dict["hover_text"],
        mode='markers',
        name = "*O scaling",
        legendgroup="*O scaling",
        showlegend=False,
        marker=dict(
            size=marker_size,
            symbol=plot_settings_dict["marker_symbol"],
            color = color_list[1],
            line = dict(
                color = plot_settings_dict["marker_line"],
                width = 2,
            )
        )
    )
    trace_list.append(trace_i)

## Ideal OOH and O vs OH Scaling

In [33]:
# Ideal OOH vs OH Scaling Line
x = line_x_array
trace_ooh_scaling = go.Scatter(
    x = x,
    y = [x[0] + 3.2, x[1] + 3.2],
    name = "ooh v oh (ideal)",
    mode='lines',
    opacity=0.6,
    line = {
        "dash": "dot",
        "color": color_list[0],
    },
)
trace_list.append(trace_ooh_scaling)

# Ideal O vs OH Scaling Line
x = line_x_array
trace_o_scaling = go.Scatter(
    x = x,
    y = [2 * x[0], 2 * x[1]],
    name = "o v oh (ideal)",
    mode='lines',
    opacity=0.6,
    line = {
        "dash": "dot",
        "color": color_list[1],
    },
)
trace_list.append(trace_o_scaling)

## Fit Lines

Before fitting the scaling relations, bad data should be diregarded.
This includes the following situations:
* Desorption: The adsorbate molecule does not bind with the surface
* Dissociation: The adsorbate is "broken up" by the surface

In [34]:
df_regr = df_master.copy()

df_regr = df_regr[df_regr["o-notes"] != "Desorbed"]
df_regr = df_regr[df_regr["oh-notes"] != "Desorbed"]
df_regr = df_regr[df_regr["ooh-notes"] != "Desorbed"]

df_regr = df_regr[df_regr["o-notes"] != "Dissociated"]
df_regr = df_regr[df_regr["oh-notes"] != "Dissociated"]
df_regr = df_regr[df_regr["ooh-notes"] != "Dissociated"]

In [35]:
df_regr_ooh_v_oh = df_regr.copy()
df = df_regr_ooh_v_oh
# Removing row with NA values for OOH or OH
df = df[np.isfinite(df["ooh"])]
df = df[np.isfinite(df["oh"])]

lr = LinearRegression()

y_data = np.array(df["ooh"].tolist())
x_data = np.array(df["oh"].tolist()).reshape(-1, 1)

lr.fit(x_data, y_data)

b = lr.intercept_
m = lr.coef_[0]
print("OOH Fit")
print("intercept (b) and slope (m) coefficients:")
print(b)
print(m)

x_array = np.array(line_x_array)
y_pred = lr.predict(x_array.reshape(-1, 1))

trace_ooh_fit = go.Scatter(
    x = x_array,
    y = y_pred,
    name = "*OOH vs *OH (fitted)",
    mode='lines',
    line = {
        "color": color_list[0],
    },
)

trace_list.append(trace_ooh_fit)

# **************************************************

df_regr_o_v_oh = df_regr.copy()
df = df_regr_o_v_oh
# Removing row with NA values for OOH or OH
df = df[np.isfinite(df["o"])]
df = df[np.isfinite(df["oh"])]

lr = LinearRegression()

y_data = np.array(df["o"].tolist())
x_data = np.array(df["oh"].tolist()).reshape(-1, 1)

lr.fit(x_data, y_data)

b = lr.intercept_
m = lr.coef_[0]
print("O Fit")
print("intercept (b) and slope (m) coefficients:")
print(b)
print(m)


x_array = np.array(line_x_array)
y_pred = lr.predict(x_array.reshape(-1, 1))

trace_o_fit = go.Scatter(
    x = x_array,
    y = y_pred,
    name = "*O vs *OH (fitted))",
    mode='lines',
    line = {
        "color": color_list[1],
    },
)

trace_list.append(trace_o_fit)

OOH Fit
intercept (b) and slope (m) coefficients:
2.917714890331197
1.298834518086638
O Fit
intercept (b) and slope (m) coefficients:
0.5017142485515365
1.8144171589792406


In [36]:
layout = dict(
    title = 'OOH, OH, O Scaling Relations',
   
    xaxis=dict(
        title="*OH Binding Energy [eV]",
        ),

    yaxis=dict(
        title="*OOH & *O Binding Energy [eV]",
        ),

#     width=200 * 4.,
#     height=200 * 3.,
)

In [37]:
border_coord = {
    "x0": 0.01,
    "y0": 0.8,
    "x1": 0.35,
    "y1": 1.,
    }

In [38]:
shapes=[
    # Rectangle reference to the axes
    
    # Border Rectangle
    {
        'type': 'rect',
        'xref': 'paper',
        'yref': 'paper',
        'x0': border_coord["x0"],
        'y0': border_coord["y0"],
        'x1': border_coord["x1"],
        'y1': border_coord["y1"],
        'line': {
            'color': 'black',
            'width': 3,
        },
#             'fillcolor': 'rgba(50, 171, 96, 0.6)',
    },

    
    # Square 1
    {
        'type': 'rect',
        'xref': 'paper',
        'yref': 'paper',
        'x0': border_coord["x0"] + 0.01,
        'y0': border_coord["y1"] - 0.05,
        'x1': border_coord["x0"] + 0.01 + 0.03,
        'y1': border_coord["y1"] - 0.01,
        'line': {
            'color': 'black',
            'width': 0.,
        },
        'fillcolor': border_color_list[0],
    },

    # Square 2
    {
        'type': 'rect',
        'xref': 'paper',
        'yref': 'paper',
        'x0': border_coord["x0"] + 0.01,
        'y0': border_coord["y1"] - 0.05 - 0.04,
        'x1': border_coord["x0"] + 0.01 + 0.03,
        'y1': border_coord["y1"] - 0.01 - 0.04,
        'line': {
            'color': 'black',
            'width': 0.,
        },
        'fillcolor': border_color_list[1],
    },

    # Square 3
    {
        'type': 'rect',
        'xref': 'paper',
        'yref': 'paper',
        'x0': border_coord["x0"] + 0.01,
        'y0': border_coord["y1"] - 0.05 - 2. * 0.04,
        'x1': border_coord["x0"] + 0.01 + 0.03,
        'y1': border_coord["y1"] - 0.01 - 2. * 0.04,
        'line': {
            'color': 'black',
            'width': 0.,
        },
        'fillcolor': border_color_list[2],
    },

    # Square 4
    {
        'type': 'rect',
        'xref': 'paper',
        'yref': 'paper',
        'x0': border_coord["x0"] + 0.01,
        'y0': border_coord["y1"] - 0.05 - 3. * 0.04,
        'x1': border_coord["x0"] + 0.01 + 0.03,
        'y1': border_coord["y1"] - 0.01 - 3. * 0.04,
        'line': {
            'color': 'black',
            'width': 0.,
        },
        'fillcolor': border_color_list[3],
    },

]

In [39]:
# Annotations     
        
annotations=[
    dict(
        x=border_coord["x0"] + 0.01 + 0.03,
        y=border_coord["y1"] - 0.05 - 0.04 + 0.087,
        showarrow=False,
        text='N-doped Graphene on Fe',
        xref='paper',
        yref='paper'
        ),

    dict(
        x=border_coord["x0"] + 0.01 + 0.03,
        y=border_coord["y1"] - 0.05 - 0.04 + 0.087 - 0.04,
        showarrow=False,
        text='Graphene on Fe',
        xref='paper',
        yref='paper'
        ),

    dict(
        x=border_coord["x0"] + 0.01 + 0.03,
        y=border_coord["y1"] - 0.05 - 0.04 + 0.087 - 2. * 0.04,
        showarrow=False,
        text='Graphene',
        xref='paper',
        yref='paper'
        ),

    dict(
        x=border_coord["x0"] + 0.01 + 0.03,
        y=border_coord["y1"] - 0.05 - 0.04 + 0.087 - 3. * 0.04,
        showarrow=False,
        text='FCC Fe(111)',
        xref='paper',
        yref='paper'
        ),
    ]


In [40]:
# layout["shapes"] = shapes
# layout["annotations"] = annotations

fig = dict(data=trace_list, layout=layout)
# py.iplot(fig, filename='01_fe_graph/pl_ooh_oh_scaling_2')
py.iplot(fig, filename='__temp__/pl_ooh_oh_scaling_2')

# plot_url = py.plot(fig, filename='01_fe_graph/my_plot')
# print(plot_url)

## Legend Color Map

In [41]:
supp_atom_r = 1.56 / 2
supp_bond_d = 2.019496967
supp_dim = (7, 7)
supp_latt_angle = 30.  # Angle between lattice vectors of surface

graph_atom_r = 0.7 / 2
# graph_bond_d = 1.42
graph_bond_d = 1.35
graph_dim = (8, 8)

In [42]:
# import svgwrite
# from IPython.display import SVG

# file_name = "an_scaling_out/color_legend.svg"
# dwg = svgwrite.Drawing(file_name, profile='tiny')
# dwg.viewbox(width=200 , height=200)

# box_height = 10
# box_width = 10
# up_left_corner = (0., 0.)

# dwg.add(dwg.rect(insert=(up_left_corner[0], up_left_corner[1] + 0 * box_height), size=(box_width, box_height), fill=border_color_list[0]))
# dwg.add(dwg.rect(insert=(up_left_corner[0], up_left_corner[1] + 1 * box_height), size=(box_width, box_height), fill=border_color_list[1]))
# dwg.add(dwg.rect(insert=(up_left_corner[0], up_left_corner[1] + 2 * box_height), size=(box_width, box_height), fill=border_color_list[2]))
# dwg.add(dwg.rect(insert=(up_left_corner[0], up_left_corner[1] + 3 * box_height), size=(box_width, box_height), fill=border_color_list[3]))

# y_offset = -1.4

# dwg.add(dwg.text("N-doped Graphene on Fe",
#     x=[up_left_corner[0] + box_width], y=[up_left_corner[1] + 1 * box_height + y_offset],
#     fill=border_color_list[0],
#     font_size='0.6em',
# #     font_family="Open Sans",
# #     style="font-size:10px; font-family:Arial",
#     ),
# )

# dwg.add(dwg.text("Graphene on Fe",
#     x=[up_left_corner[0] + box_width], y=[up_left_corner[1] + 2 * box_height + y_offset],
#     fill=border_color_list[1],
#     font_size='0.6em'),
#     )
# dwg.add(dwg.text("Graphene",
#     x=[up_left_corner[0] + box_width], y=[up_left_corner[1] + 3 * box_height + y_offset],
#     fill=border_color_list[2],
#     font_size='0.6em'),
#     )
# dwg.add(dwg.text("FCC Fe(111)",
#     x=[up_left_corner[0] + box_width], y=[up_left_corner[1] + 4 * box_height + y_offset],
#     fill=border_color_list[3],
#     font_size='0.6em'),
    
#     )

# dwg.save()


# SVG(filename=file_name)